# Handling NULL Values using Spark SQL

In [0]:
%sql

select 1 + null as result

result
null


In [0]:
%sql

select concat('Hello', null)

"concat(Hello, NULL)"
null


In [0]:
%sql

desc function nvl

function_desc
Function: nvl
Class: org.apache.spark.sql.catalyst.expressions.Nvl
"Usage: nvl(expr1, expr2) - Returns `expr2` if `expr1` is null, or `expr1` otherwise."


In [0]:
%sql

desc function coalesce

function_desc
Function: coalesce
Class: org.apache.spark.sql.catalyst.expressions.Coalesce
"Usage: coalesce(expr1, expr2, ...) - Returns the first non-null argument if exists. Otherwise, null."


In [0]:
%sql

desc function nvl2

function_desc
Function: nvl2
Class: org.apache.spark.sql.catalyst.expressions.Nvl2
"Usage: nvl2(expr1, expr2, expr3) - Returns `expr2` if `expr1` is not null, or `expr3` otherwise."


In [0]:
%sql
select nvl(1, 0) nvl, coalesce(1, 0) coalesce

nvl,coalesce
1,1


In [0]:
%sql
select nvl(null, 0) nvl, coalesce(null, 0) as coalesce

nvl,coalesce
0,0


In [0]:
%sql

select coalesce(null, null, 2, null, 3) as re

re
2


In [0]:
%sql

use itversity_retail

In [0]:
%sql

drop table if exists sales

In [0]:
%sql
create table if not exists sales (
  sales_id int,
  sales_amount float,
  commisstion_pct int
)

In [0]:
%sql

insert into sales values
  (1, 1000, 10),
  (2, 1500, 8),
  (3, 500, null),
  (4, 800, 5),
  (5, 250, null)

num_affected_rows,num_inserted_rows
5,5


In [0]:
%sql

select * from sales

sales_id,sales_amount,commisstion_pct
1,1000.0,10
2,1500.0,8
3,500.0,null
4,800.0,5
5,250.0,null


In [0]:
%sql
select s.*,
nvl(commisstion_pct,0) as commisstion_pct2
from sales s

sales_id,sales_amount,commisstion_pct,commisstion_pct2
1,1000.0,10,10
2,1500.0,8,8
3,500.0,null,0
4,800.0,5,5
5,250.0,null,0


In [0]:
%sql
select s.*,
round(s.sales_amount * s.commisstion_pct / 100,2) as incorrect_commission_amount
from sales s

sales_id,sales_amount,commisstion_pct,incorrect_commission_amount
1,1000.0,10,100.0
2,1500.0,8,120.0
3,500.0,null,null
4,800.0,5,40.0
5,250.0,null,null


In [0]:
%sql
select s.*,
round(s.sales_amount * nvl(s.commisstion_pct,0) / 100,2) as commission_amount,
s.sales_amount - round(round(s.sales_amount * s.commisstion_pct / 100,2)) as sales_revenue
from sales s

sales_id,sales_amount,commisstion_pct,commission_amount,sales_revenue
1,1000.0,10,100.0,900.0
2,1500.0,8,120.0,1380.0
3,500.0,null,0.0,null
4,800.0,5,40.0,760.0
5,250.0,null,0.0,null


In [0]:
%sql
select s.*,
round(s.sales_amount * coalesce(s.commisstion_pct,0) / 100,2) as commission_amount,
s.sales_amount - round(s.sales_amount * coalesce(s.commisstion_pct,0) / 100,2) as sales_revenue
from sales s

sales_id,sales_amount,commisstion_pct,commission_amount,sales_revenue
1,1000.0,10,100.0,900.0
2,1500.0,8,120.0,1380.0
3,500.0,null,0.0,500.0
4,800.0,5,40.0,760.0
5,250.0,null,0.0,250.0


In [0]:
%sql

-- Add 1% if commisstion_pct is not null
-- If commisstion_pct is null, set it to 2%
select s.*,
nvl2(s.commisstion_pct, s.commisstion_pct + 1, 2) as commisstion_pct
from sales s

sales_id,sales_amount,commisstion_pct,commisstion_pct
1,1000.0,10,11
2,1500.0,8,9
3,500.0,null,2
4,800.0,5,6
5,250.0,null,2


In [0]:
%sql

-- round(s.sales_amount + nvl2(s.commisstion_pct, s.commisstion_pct + 1, 2) / 100, 2) as comission_amount
-- round(s.sales_amount * nvl2(s.commisstion_pct, s.commisstion_pct + 1, 2) / 100, 2) as comission_amount

select s.*,
round(s.sales_amount * nvl2(s.commisstion_pct, s.commisstion_pct + 1, 2) / 100, 2) as comission_amount
from sales s

sales_id,sales_amount,commisstion_pct,comission_amount
1,1000.0,10,110.0
2,1500.0,8,135.0
3,500.0,null,10.0
4,800.0,5,48.0
5,250.0,null,5.0


In [0]:
%sql

select s.*,
  case 
    when s.commisstion_pct is null then 2
    else s.commisstion_pct + 1
  end as commisstion_pct2
from sales s

sales_id,sales_amount,commisstion_pct,commisstion_pct2
1,1000.0,10,11
2,1500.0,8,9
3,500.0,null,2
4,800.0,5,6
5,250.0,null,2


In [0]:
%sql

select s.*,
  case 
    when s.commisstion_pct is null then round((s.sales_amount * 2 / 100) , 2)
    else round((s.sales_amount * (s.commisstion_pct + 1) / 100) , 2)
  end as commisstion_pct2
from sales s

sales_id,sales_amount,commisstion_pct,commisstion_pct2
1,1000.0,10,110.0
2,1500.0,8,135.0
3,500.0,null,10.0
4,800.0,5,48.0
5,250.0,null,5.0
